In [1]:
import org.apache.spark.graphx._
def readRdf(sc:org.apache.spark.SparkContext, filename:String) = {
  val r = sc.textFile(filename).map(_.split("\t"))
  val v = r.map(_(1)).union(r.map(_(3))).distinct.zipWithIndex
  Graph(v.map(_.swap),
        r.map(x => (x(1),(x(2),x(3))))
         .join(v)
         .map(x => (x._2._1._2,(x._2._2,x._2._1._1)))
         .join(v)
         .map(x => new Edge(x._2._1._1, x._2._2, x._2._1._2)))
}

In [ ]:
val gf = readRdf(sc, "/data/yagoFacts.tsv").subgraph(_.attr == "<exports>")
val e = gf.edges.map(e => Edge(e.srcId, e.dstId, 1.0))
val (gs,mean) = lib.SVDPlusPlus.run(e,
      new lib.SVDPlusPlus.Conf(2,10,0,5,0.007,0.007,0.005,0.015))
